# Calculations in $\frac{\mathbb{Z}[t]}{(|tI -M_{1}|)}$

### Modules

In [1]:
#import os
#import numpy as np
import sympy as sp
sp.init_printing(use_latex='mathjax')
#import pandas as pd
#import ast
import itertools as itr

#from multiprocessing import Pool, cpu_count
#from tqdm.notebook import tqdm

#from snappy import *
#S = twister.Surface('S_2_1')

## Symplectic representation

In [2]:
K = sp.MatrixSymbol('K',2,2) #([[0,-1],[0,0]])
L = sp.MatrixSymbol('L',2,2) #([[1,0],[1,1]])
I = sp.Identity(2)
O = sp.ZeroMatrix(2,2)
J = sp.MatrixSymbol('J',2,2) #([[0,1],[-1,0]])

Ma = sp.BlockMatrix([[L.inv(), O], [O, I]])
Mb = sp.BlockMatrix([[L.transpose(), K], [K, L.transpose()]])
Mc = sp.BlockMatrix([[I, O], [O, L.inv()]])
Md = sp.BlockMatrix([[I, O], [O, L.transpose()]])
#display(Ma, Mb, Mc, Md)

MJ = sp.BlockMatrix([[J, O], [O, J]])
#display(MJ)

### Functions

In [3]:
def ev(M):
    return M.subs([
        (L, sp.Matrix([[1,0],[1,1]])),
        (K, sp.Matrix([[0,-1],[0,0]])),
        (J, sp.Matrix([[0,1],[-1,0]])),
    ]).as_explicit()
#evMd = ev(Md); display(evMd, evMd.det())

def is_Sp(M):
    if type(M) == sp.matrices.expressions.blockmatrix.BlockMatrix:
        return sp.block_collapse(ev(M).transpose()*ev(MJ)*ev(M)) == ev(MJ)
    else:
        return M.transpose()*ev(MJ)*M == ev(MJ)

#for M in {Ma, Mb, Mc, Md}:
#    if is_Sp(M):
#        print(f"{M} is in Sp(4,Z)")

### The symplectic images of Abcd and abcD

In [4]:
M_Abcd = sp.block_collapse((Ma.inv())*Mb*Mc*Md)
M1 = ev(M_Abcd)
display(M_Abcd, M1)
M_abcD = sp.block_collapse(Ma*Mb*Mc*(Md.inv()))
M2 = ev(M_abcD)
display(M_abcD, M2)

for M in {M1,M2}:
    print(f"is_Sp? --> {is_Sp(M)}")

⎡   T       -1  T⎤
⎢L⋅L   L⋅K⋅L  ⋅L ⎥
⎢                ⎥
⎢       T  -1  T ⎥
⎣ K    L ⋅L  ⋅L  ⎦

⎡1  1   1   0⎤
⎢            ⎥
⎢1  2   1   0⎥
⎢            ⎥
⎢0  -1  0   1⎥
⎢            ⎥
⎣0  0   -1  0⎦

⎡                      -1⎤
⎢ -1  T   -1    -1 ⎛ T⎞  ⎥
⎢L  ⋅L   L  ⋅K⋅L  ⋅⎝L ⎠  ⎥
⎢                        ⎥
⎢                    -1  ⎥
⎢          T  -1 ⎛ T⎞    ⎥
⎣  K      L ⋅L  ⋅⎝L ⎠    ⎦

⎡1   1   1   -2⎤
⎢              ⎥
⎢-1  0   -1  2 ⎥
⎢              ⎥
⎢0   -1  0   1 ⎥
⎢              ⎥
⎣0   0   -1  2 ⎦

is_Sp? --> True
is_Sp? --> True


### The charactoristic polynomial of M1 (M2)

In [5]:
cp_M1 = M1.charpoly().as_expr()
cp_M2 = M2.charpoly().as_expr()

print(f"cp_M1 == cp_M2 ?   --> {cp_M1==cp_M2}")

display(cp_M1.factor())

cp_M1 == cp_M2 ?   --> True


 4      3      2          
λ  - 3⋅λ  + 3⋅λ  - 3⋅λ + 1

### The rational form of M1 (M2)

In [8]:
C = sp.matrices.expressions.CompanionMatrix(M1.charpoly()).as_explicit()
display(C)
print(f"is_Sp? --> {is_Sp(C)}")

⎡0  0  0  -1⎤
⎢           ⎥
⎢1  0  0  3 ⎥
⎢           ⎥
⎢0  1  0  -3⎥
⎢           ⎥
⎣0  0  1  3 ⎦

is_Sp? --> False


## In $\mathbb{Z}[t]$

### Functions

In [37]:
def poly(v,A):
    terms = [v[i]*(A**i) for i in range(len(v))]
    return v[0]*(A**0)+v[1]*(A**1)+v[2]*(A**2)+v[3]*(A**3)

In [38]:
c = sp.symbols('c:4')

Cc = sp.Matrix(0,0,[])
for i in range(4):
    v = (C**i)*sp.Matrix(4,1,c)
    Cc = Cc.row_join(v)
    
display(Cc)

⎡c₀     -c₃        -c₂ - 3⋅c₃        -c₁ - 3⋅c₂ - 6⋅c₃    ⎤
⎢                                                         ⎥
⎢c₁  c₀ + 3⋅c₃    3⋅c₂ + 8⋅c₃       3⋅c₁ + 8⋅c₂ + 15⋅c₃   ⎥
⎢                                                         ⎥
⎢c₂  c₁ - 3⋅c₃  c₀ - 3⋅c₂ - 6⋅c₃    -3⋅c₁ - 6⋅c₂ - 10⋅c₃  ⎥
⎢                                                         ⎥
⎣c₃  c₂ + 3⋅c₃  c₁ + 3⋅c₂ + 6⋅c₃  c₀ + 3⋅c₁ + 6⋅c₂ + 12⋅c₃⎦

In [39]:
f = cp_M1.subs('lambda', t)
display(sp.collect(f,t))

 4      3      2          
t  - 3⋅t  + 3⋅t  - 3⋅t + 1

In [56]:
d = sp.symbols('d:4')
h = sp.Matrix(4,1,[d[i] for i in range(4)])

Cch = Cc*h
display(Cch)

t = sp.symbols('t')
exp = sp.Poly((poly(c,t)*poly(d,t)), t)

while exp.degree() >= 4:
    Q, R = exp.div((t**4).as_poly())
    exp = Q*(t**4-f) + R
    
for i in range(1,4):
    cf = exp.as_expr().coeff(t**i)
    #display(cf, Cch[i].as_expr().expand())
    print(cf == Cch[i].as_expr().expand())

⎡           c₀⋅d₀ - c₃⋅d₁ + d₂⋅(-c₂ - 3⋅c₃) + d₃⋅(-c₁ - 3⋅c₂ - 6⋅c₃)           ⎤
⎢                                                                              ⎥
⎢     c₁⋅d₀ + d₁⋅(c₀ + 3⋅c₃) + d₂⋅(3⋅c₂ + 8⋅c₃) + d₃⋅(3⋅c₁ + 8⋅c₂ + 15⋅c₃)     ⎥
⎢                                                                              ⎥
⎢  c₂⋅d₀ + d₁⋅(c₁ - 3⋅c₃) + d₂⋅(c₀ - 3⋅c₂ - 6⋅c₃) + d₃⋅(-3⋅c₁ - 6⋅c₂ - 10⋅c₃)  ⎥
⎢                                                                              ⎥
⎣c₃⋅d₀ + d₁⋅(c₂ + 3⋅c₃) + d₂⋅(c₁ + 3⋅c₂ + 6⋅c₃) + d₃⋅(c₀ + 3⋅c₁ + 6⋅c₂ + 12⋅c₃)⎦

True
True
True
